In [9]:
class MovingAverage():
    def __init__(self, pts):
        self._pts = [0]*pts
        self._npts = pts
        self._cur = 0
        self._sum = 0
        
    def add(self, pt):
        cp = self._pts[self._cur]
        self._pts[self._cur] = pt
        self._inc()
        self._sum = self._sum + pt - cp
    
    def _inc(self):
        self._cur += 1
        if self._cur >= self._npts:
            self._cur = 0
            
    def ave(self):
        return self._sum / self._npts
    
    def process(self, pt):
        self.add(pt)
        return self.ave()
    
    

In [27]:
from time import sleep
ma = MovingAverage(3)
l = [0]*23
for i, n in enumerate(range(20)):
    l[i] = n
    print(ma.process(n), sum(l[i:i+3])/3, ma._pts, l[i:i+3])
    

0.0 0.0 [0, 0, 0] [0, 0, 0]
0.3333333333333333 0.3333333333333333 [0, 1, 0] [1, 0, 0]
1.0 0.6666666666666666 [0, 1, 2] [2, 0, 0]
2.0 1.0 [3, 1, 2] [3, 0, 0]
3.0 1.3333333333333333 [3, 4, 2] [4, 0, 0]
4.0 1.6666666666666667 [3, 4, 5] [5, 0, 0]
5.0 2.0 [6, 4, 5] [6, 0, 0]
6.0 2.3333333333333335 [6, 7, 5] [7, 0, 0]
7.0 2.6666666666666665 [6, 7, 8] [8, 0, 0]
8.0 3.0 [9, 7, 8] [9, 0, 0]
9.0 3.3333333333333335 [9, 10, 8] [10, 0, 0]
10.0 3.6666666666666665 [9, 10, 11] [11, 0, 0]
11.0 4.0 [12, 10, 11] [12, 0, 0]
12.0 4.333333333333333 [12, 13, 11] [13, 0, 0]
13.0 4.666666666666667 [12, 13, 14] [14, 0, 0]
14.0 5.0 [15, 13, 14] [15, 0, 0]
15.0 5.333333333333333 [15, 16, 14] [16, 0, 0]
16.0 5.666666666666667 [15, 16, 17] [17, 0, 0]
17.0 6.0 [18, 16, 17] [18, 0, 0]
18.0 6.333333333333333 [18, 19, 17] [19, 0, 0]
